In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler, autocast
import numpy as np


In [ ]:
# Data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((160, 160)),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder("dataset/", transform=transform)
val_dataset = datasets.ImageFolder("dataset/", transform=transform)
test_dataset = datasets.ImageFolder("dataset/", transform=transform)

# DataLoaders with reduced batch size
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)


In [3]:
class ConvMixer(nn.Module):
    def __init__(self, dim, depth, num_classes):
        super(ConvMixer, self).__init__()
        self.dim = dim
        self.depth = depth

        # Initial Patch Embedding
        self.patch_embed = nn.Sequential(
            nn.Conv2d(3, dim, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm2d(dim),
            nn.GELU()
        )

        # ConvMixer Layers
        self.convmixer_blocks = nn.Sequential(*[
            nn.Sequential(
                nn.Conv2d(dim, dim, kernel_size=7, stride=1, groups=dim, padding=3),  # Depthwise Convolution
                nn.BatchNorm2d(dim),
                nn.GELU(),
                nn.Conv2d(dim, dim, kernel_size=1),  # Pointwise Convolution
                nn.BatchNorm2d(dim),
                nn.GELU()
            ) for _ in range(depth)
        ])

        # Adaptive Pooling and Classification Head
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(dim, num_classes)

    def forward(self, x):
        x = self.patch_embed(x)
        x = self.convmixer_blocks(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize ConvMixer with reduced complexity
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvMixer(dim=128, depth=6, num_classes=30).to(device)  # Reduced `dim` and `depth`
print(model)


ConvMixer(
  (patch_embed): Sequential(
    (0): Conv2d(3, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
  )
  (convmixer_blocks): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
      (3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): GELU(approximate='none')
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU(approximate='none')
      (3): Conv2d(128, 128, kernel_size=(1, 1)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)


In [5]:
# Initialize GradScaler for mixed precision
scaler = GradScaler()

# Training function
def train_convmixer(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():  # Enable mixed precision
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Backpropagation
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(loader)
    train_accuracy = 100 * correct / total
    return train_loss, train_accuracy

# Validation function
def validate_convmixer(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = running_loss / len(loader)
    val_accuracy = 100 * correct / total
    return val_loss, val_accuracy


C:\Users\anirv\AppData\Local\Temp\ipykernel_25876\3322041970.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [6]:
# Training loop
num_epochs = 30
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

torch.cuda.empty_cache()  # Clear GPU memory before starting

for epoch in range(num_epochs):
    train_loss, train_accuracy = train_convmixer(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = validate_convmixer(model, val_loader, criterion)
    scheduler.step()

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# Save the model
torch.save(model.state_dict(), 'optimized_convmixer.pth')
print("Model saved!")


C:\Users\anirv\AppData\Local\Temp\ipykernel_25876\3322041970.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Enable mixed precision


Epoch 1/30
Train Loss: 0.8447, Train Accuracy: 69.71%
Val Loss: 0.8050, Val Accuracy: 70.00%
Epoch 2/30
Train Loss: 0.8043, Train Accuracy: 70.00%
Val Loss: 0.8040, Val Accuracy: 70.00%
Epoch 3/30
Train Loss: 0.8017, Train Accuracy: 70.00%
Val Loss: 0.7966, Val Accuracy: 70.00%
Epoch 4/30
Train Loss: 0.8006, Train Accuracy: 70.00%
Val Loss: 0.7958, Val Accuracy: 70.00%
Epoch 5/30
Train Loss: 0.7983, Train Accuracy: 70.00%
Val Loss: 0.7969, Val Accuracy: 70.00%


KeyboardInterrupt: 

In [ ]:
# Evaluate the model on test data
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Classification Report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=train_dataset.classes))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.classes)
disp.plot(cmap=plt.cm.Blues, xticks_rotation="vertical")
plt.title("Confusion Matrix")
plt.show()


NameError: name 'model' is not defined

In [9]:



# Clear the GPU cache
torch.cuda.empty_cache()